In [2]:
import numpy as np

import sys
sys.path.append('../utils')
from GridWorld import get_standard_grid, ACTIONS

In [3]:
THRESHOLD = 1e-3
GAMMA = 0.9

def printValues(values, g):
    # values are a dictionary of tuples with the value being the probability
    # g is the gridWorld
    for i in range(g.rows):
        print("-------------------------")
        for j in range(g.cols):
            v = values.get((i, j), 0)
            if v >= 0:
                print(" %.2f|" % v, end="")
            else:
                print("%.2f|" % v, end="")
        print("")
    

def printPolicy(policy, g):
    for i in range(g.rows):
        print("-------------------------")
        for j in range(g.cols):
            p = policy.get((i, j), ' ')
            print(" %s |" % p, end="")
        print("")

def get_transition_probs_rewards(grid):
    transition_probs = {}
    rewards = {}
    for i in range(grid.rows):
        for j in range(grid.cols):
            s = (i, j)
            if not grid.is_terminal(s):
                for a in ACTIONS:
                    s2 = grid.get_next_state(a, s)
                    transition_probs[(s, a, s2)] = 1
                    if s2 in grid.rewards:
                        rewards[(s, a, s2)] = grid.rewards[s2]
    return transition_probs, rewards

def evaluate_deterministic_policy(grid, policy):
    V = {}
    for s in grid.all_states():
        V[s] = 0

    # discount factor
    gamma = 0.9

    it = 0
    while True:
        biggest_change = 0
        for s in grid.all_states():
            if not grid.is_terminal(s):
                old_v = V[s]
                new_v = 0 # we will accumulate the answer

                for a in ACTIONS:
                    for s2 in grid.all_states():
                        action_prob = 1 if policy.get(s) == a else 0

                        r = rewards.get((s, a, s2), 0)
                        new_v += action_prob * transition_probs.get((s, a, s2), 0) * (r + gamma * V[s2])

                # after getting the new value we update the table
                V[s] = new_v
                biggest_change = max(biggest_change, np.abs(old_v - V[s]))          
                
        it += 1

        if biggest_change < THRESHOLD:
            break

    return V

In [4]:
grid = get_standard_grid()
transition_probs, rewards = get_transition_probs_rewards(grid)

print("rewards:")
printValues(grid.rewards, grid)

policy = {}
for s in grid.actions.keys():
    policy[s] = np.random.choice(ACTIONS)

print("initial policy:")
printPolicy(policy, grid)

# repeat until convergence
while True:
    # policy evaluation first
    V = evaluate_deterministic_policy(grid, policy)

    # policy improvement
    is_policy_converged = True
    for s in grid.actions.keys():
        old_a = policy[s]
        new_a = None
        best_value = float('-inf')

        # loop through all possible actions to find the best
        for a in ACTIONS:
            v = 0
            for s2 in grid.all_states():
                r = rewards.get((s, a, s2), 0)
                # bellman equation
                v += transition_probs.get((s, a, s2), 0) * (r + GAMMA * V[s2])

            if v > best_value:
                best_value = v
                new_a = a

        policy[s] = new_a
        if new_a != old_a:
            is_policy_converged = False

    if is_policy_converged:
        break

print("values:")
printValues(V, grid)
print("policy:")
printPolicy(policy, grid)

rewards:
-------------------------
 0.00| 0.00| 0.00| 1.00|
-------------------------
 0.00| 0.00| 0.00|-1.00|
-------------------------
 0.00| 0.00| 0.00| 0.00|
initial policy:
-------------------------
 D | U | D |   |
-------------------------
 U |   | U |   |
-------------------------
 L | L | D | L |
values:
-------------------------
 0.81| 0.90| 1.00| 0.00|
-------------------------
 0.73| 0.00| 0.90| 0.00|
-------------------------
 0.66| 0.73| 0.81| 0.73|
policy:
-------------------------
 R | R | R |   |
-------------------------
 U |   | U |   |
-------------------------
 U | R | U | L |
